# This Notebook was written to test my knowledge regarding basline and PPO on Cart POle environemnt

In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
!pip install stable-baselines3[extra]

     -------------------------------------- 177.2/177.2 kB 3.6 MB/s eta 0:00:00
     --------------------------------------- 10.4/10.4 MB 29.7 MB/s eta 0:00:00
     ------------------------------------- 162.2/162.2 MB 13.3 MB/s eta 0:00:00
  Using cached gym-0.21.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.2/7.2 MB 21.0 MB/s eta 0:00:00
     ------------------------------------- 904.7/904.7 kB 28.9 MB/s eta 0:00:00
     ------------------------------------- 895.7/895.7 kB 28.6 MB/s eta 0:00:00
  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
     ---------------------------------------- 5.9/5.9 MB 19.7 MB/s eta 0:00:00
     --------------------------------------- 35.6/35.6 MB 16.8 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 4.5 MB/s eta 0:00:00
  Using cached AutoROM.accept-rom-license-0.4.2.tar.gz (9.8 kB)
  Installing build d

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym-robotics 1.0.0 requires gym>=0.26, but you have gym 0.21.0 which is incompatible.


In [1]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name)
# only in 0.21 make does not take into account the "render_mode" argument

In [10]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:41.0
Episode:2 Score:12.0
Episode:3 Score:11.0
Episode:4 Score:21.0
Episode:5 Score:16.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [11]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

0

In [12]:
# [cart position, cart velocity, pole angle(in radian), pole angular velocity]
env.observation_space.sample()

array([-2.4352672e+00,  1.0403606e+38,  1.1980361e-02, -1.6307510e+38],
      dtype=float32)

# 3. Train an RL Model

In [4]:
# Make your directories first
log_path = os.path.join('Training','Logs')

In [5]:
log_path

'Training\\Logs'

In [15]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log=log_path)

Using cuda device


In [17]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 613  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 510          |
|    iterations           | 2            |
|    time_elapsed         | 8            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0039189965 |
|    clip_fraction        | 0.0208       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.558       |
|    explained_variance   | 0.281        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.369        |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.00316     |
|    value_loss           | 6.48         |
----------------------------

# 4. Save and Reload Model

In [6]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [19]:
model.save(PPO_path)

In [20]:
del model

In [7]:
model = PPO.load(PPO_path, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# 4. Evaluation

In [8]:
from stable_baselines3.common.evaluation import evaluate_policy

In [9]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\ProgramData\Anaconda3\envs\om\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [10]:
env.close()

In [ ]:
# average reward of 200 is max 0 is the standard deviation of the rewards.

# 5. Test Model

In [11]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info {'TimeLimit.truncated': True}


In [12]:
env.close()

# 6. Viewing Logs in Tensorboard

In [13]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [15]:
training_log_path

'Training\\Logs\\PPO_2'

In [16]:
!tensorboard --logdir={training_log_path}

^C


# 7. Adding a callback to the training Stage

In [17]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [18]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [20]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [21]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [22]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cuda device


In [23]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 555  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 475         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009431087 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00433    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.75        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0167     |
|    value_loss           | 55.5        |
-----------------------------------------
---

In [24]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [25]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(192.6, 22.2)

In [26]:
env.close()

# 8. Changing Policies

In [27]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [28]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cuda device


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 553  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 470         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014398373 |
|    clip_fraction        | 0.241       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.000492   |
|    learning_rate        | 0.0003      |
|    loss                 | 4.02        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0281     |
|    value_loss           | 18.9        |
-----------------------------------------
----------------------------------

In [30]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [31]:
env.close()

# 9. Using an Alternate Algorithm

In [32]:
from stable_baselines3 import DQN

In [33]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cuda device


In [34]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.962    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5641     |
|    time_elapsed     | 0        |
|    total_timesteps  | 79       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.919    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 7433     |
|    time_elapsed     | 0        |
|    total_timesteps  | 171      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.883    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 7455     |
|    time_elapsed     | 0        |
|    total_timesteps  | 246      |
----------------------------------
------------------------

In [35]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [36]:
model.save(dqn_path)

In [37]:
model = DQN.load(dqn_path, env=env)

In [38]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.9, 0.7000000000000001)

In [39]:
env.close()